In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
import seaborn as sb     # this is for heatmaps
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
transform = transforms.ToTensor()
train_data = datasets.CIFAR10(root='../data/cnn_files/CIFAR10', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root='../data/cnn_files/CIFAR10', train=False, download=True, transform=transform)

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
torch.manual_seed(101)
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

In [ ]:
######################## let's see what does cifar10 look like
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for images, labels in train_loader:
    break

In [ ]:
labels

In [ ]:
print(labels[:5])
print([class_names[class_index] for class_index in labels[:5]])
print(labels[5:])
print([class_names[class_index] for class_index in labels[5:]])
im = make_grid(images, nrow=5)
plt.figure(figsize=(10,10))
plt.imshow(np.transpose(im.numpy(), (1, 2, 0)))

In [ ]:
########################

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, stride=1)
        self.fc1 = nn.Linear(6*6*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, kernel_size=2, stride=2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, kernel_size=2, stride=2)
        X = X.view(-1, 6*6*16)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = F.log_softmax(self.fc3(X), dim=1)
        
        return X       

In [ ]:
torch.manual_seed(101)
model = ConvNet()
model

In [ ]:
for params in model.parameters():
    print(params.numel())

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import time
start_time = time.time()

epochs = 10
train_losses_list = []
test_losses_list = []
train_corrects_list = []
test_corrects_list = []

for i in range(epochs):
    train_correct = 0
    test_correct = 0
    
    for batch_num, (X_train, y_train) in enumerate(train_loader):
        batch_num += 1
        
        y_pred = model.forward(X_train)
        loss = criterion(y_pred, y_train)
        
        predicted = torch.max(y_pred.data, dim=1)[1]
        batch_corrects = (predicted == y_train).sum()
        train_correct += batch_corrects
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch_num % 1000 == 0:
            accuracy_till_here = (train_correct.item()/(batch_num*10))*100
            print(f'epoch: {i}, batch number:{batch_num}, loss:{loss.item()}, accuracy:{accuracy_till_here}')

    train_losses_list.append(loss)
    train_corrects_list.append(train_correct)
                                  
    with torch.no_grad():
        for batch_num, (X_test, y_test) in enumerate(test_loader):
            batch_num += 1

            y_eval = model.forward(X_test)

            predicted = torch.max(y_eval.data, dim=1)[1]
            batch_corrects = (predicted == y_test).sum()
            test_correct += batch_corrects

        loss = criterion(y_eval, y_test)
        test_losses_list.append(loss)
        test_corrects_list.append(test_correct)

duration = time.time() - start_time
print(f'Training took {duration/60} mins')

In [ ]:
torch.save(model.state_dict(), '../models/my_cnn_cifar10_1.pt')

In [ ]:
plt.plot(train_losses_list, label='train loss')
plt.plot(test_losses_list, label='validation loss')
plt.title('loss at epoch')
plt.legend()

In [ ]:
plt.plot([(t.item()/50000)*100 for t in train_corrects_list], label='train accuracy')
plt.plot([(t.item()/10000)*100 for t in test_corrects_list], label='validation accuracy')
plt.title('Accuracy at the end of each epoch')
plt.legend()

In [ ]:
test_all_loader = DataLoader(test_data, batch_size=10000, shuffle=False)
with torch.no_grad():
    for X_test, y_test in test_all_loader:
        y_val = model.forward(X_test)
        predicted = torch.max(y_val, 1)[1]
        corrects = (predicted == y_test).sum()
corrects.item()/len(X_test)  

In [ ]:
conf_mat = confusion_matrix(predicted, y_test)
df_conf_mat = pd.DataFrame(conf_mat, class_names, class_names)
plt.figure(figsize=(9,6))
sb.heatmap(df_conf_mat, annot=True, fmt='d', cmap='BuGn')
plt.xlabel('prediction')
plt.ylabel('label (ground truth)')
plt.show()